# Taking vacations on the best possible weather conditions

## Dependecies and configuration

In [1]:
# Import libraries
import pandas as pd
import requests
import os
import gmaps

# Import keys.
from api_keys import g_key

# Weather information for 1000 cities dataset file
one_thousand_cities_weather_data_path = os.path.join("output", "one_thousand_unique_cities.csv")

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Base query for Goodle nearbysearch API.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Load datasets and prepare parameters

In [2]:
# Read weather information for 1000 cities dataset.
cities_df = pd.read_csv(one_thousand_cities_weather_data_path)

# Create lists' parameters for google maps
locations = cities_df[["lat", "lon"]].astype(float)
weather_humidity = cities_df["humidity"].astype(float)


## Create map to show humidity in a 1000 cities

In [6]:
# Create a heat map showing the humidity for a list of cities

humidity_fig = gmaps.figure()

humidity_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

humidity_fig.add_layer(humidity_heat_layer)

humidity_fig


Figure(layout=FigureLayout(height='420px'))

## Look up the cities with the best weather

In [7]:
# Look up cities in the dataframe according to certain conditions

best_weather_conditions_cities = cities_df[
    (cities_df['temperature'] > 65) & (cities_df['temperature'] < 75)
    & (cities_df['wind_speed'] < 10)
    & (cities_df['cloudiness'] == 0)
]

best_weather_conditions_cities = best_weather_conditions_cities.reset_index(drop = True)
best_weather_conditions_cities

,city_id,city_name,state,country,lat,lon,temperature,humidity,cloudiness,wind_speed,weather_condition,weather_condition_desc,datetime
0,347863,Siwa Oasis,NaN,EG,29.204090,25.519520,71.51,38.0,0.0,6.22,Clear,clear sky,2020-06-18 22:54:00.454336
1,299982,Tabanlı,NaN,TR,38.755638,41.689919,65.17,45.0,0.0,0.65,Clear,clear sky,2020-06-18 22:54:10.092255
2,261211,Kaména Voúrla,NaN,GR,38.778439,22.785730,66.81,49.0,0.0,6.93,Clear,clear sky,2020-06-18 22:54:10.721681
3,6639653,Alimos,NaN,GR,37.916672,23.716669,73.69,60.0,0.0,9.93,Clear,clear sky,2020-06-18 22:54:13.670767
4,7648907,Alborz,NaN,IR,36.011169,50.809559,69.80,46.0,0.0,6.93,Clear,clear sky,2020-06-18 22:54:24.538750
5,616278,Geghakert,NaN,AM,40.185162,44.243309,66.20,68.0,0.0,3.36,Rain,light rain,2020-06-18 22:54:53.416798
6,3404921,Brejinho,NaN,BR,-7.116670,-48.049999,66.20,88.0,0.0,2.24,Clear,clear sky,2020-06-18 22:54:58.954399
7,3455447,Palmas de Monte Alto,NaN,BR,-14.267220,-43.161942,65.48,80.0,0.0,5.35,Clear,clear sky,2020-06-18 22:54:59.707117
8,3397982,Itaiçaba,NaN,BR,-4.674440,-37.822498,72.97,91.0,0.0,6.42,Clear,clear sky,2020-06-18 22:55:07.484313
9,3463590,Fátima,NaN,BR,-10.600000,-38.216671,65.05,95.0,0.0,3.76,Clear,clear sky,2020-06-18 22:55:27.862584


## Look up hotels near the cities with the best weather

In [8]:
# Interact with Google API to get data on one hotel close to each city in the list of cities with ideal weather.

# Fix parameter values to call the API
target_search = "hotel"
target_type = "lodging"
target_radius = 5000

# Parameters for the query to google nearbysearch API.
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

counter = 0

# Call the google API for each city with ideal weather conditions.
print("Starting data request...")

for index, city in best_weather_conditions_cities.iterrows() :
    counter += 1
    
    target_coordinates = f"{city['lat']},{city['lon']}"
    params.update({"location": target_coordinates})
    
    print(f"Requesting data for: {counter} - {city['city_name']}, {city['country']} (lat: {city['lat']}, lon: {city['lon']})...")
    try :
        response = requests.get(base_url, params=params)
        hotels_data = response.json()
    
        # Only get data from the cities that had a hotel within 5000
        if hotels_data['status'] == "OK" :
            best_weather_conditions_cities.loc[index, 'hotel_name'] = hotels_data['results'][0]['name']
            best_weather_conditions_cities.loc[index, 'hotel_vicinity'] = hotels_data['results'][0]['vicinity']
            print("Data received.")
        else :
            print("Data not available.")
    except:
        print(f"There was an error retrieving the hotel information for this city: {city['city_id']} - {city['city_name']}, {city['country']}")

print("Data request completed.")

# Remove all cities that didn't have a hotel within 5000 meters.
best_weather_conditions_cities.dropna(subset = ['hotel_name'], inplace = True)

Starting data request...
Requesting data for: 1 - Siwa Oasis, EG (lat: 29.20409, lon: 25.51952)...
Data received.
Requesting data for: 2 - Tabanlı, TR (lat: 38.755638, lon: 41.689919)...
Data not available.
Requesting data for: 3 - Kaména Voúrla, GR (lat: 38.778439, lon: 22.785729999999997)...
Data received.
Requesting data for: 4 - Alimos, GR (lat: 37.916672, lon: 23.716669)...
Data received.
Requesting data for: 5 - Alborz, IR (lat: 36.011169, lon: 50.809559)...
Data received.
Requesting data for: 6 - Geghakert, AM (lat: 40.185162, lon: 44.243309)...
Data received.
Requesting data for: 7 - Brejinho, BR (lat: -7.11667, lon: -48.049999)...
Data not available.
Requesting data for: 8 - Palmas de Monte Alto, BR (lat: -14.267220000000002, lon: -43.161941999999996)...
Data received.
Requesting data for: 9 - Itaiçaba, BR (lat: -4.67444, lon: -37.822497999999996)...
Data not available.
Requesting data for: 10 - Fátima, BR (lat: -10.6, lon: -38.216671000000005)...
Data not available.
Requestin

In [9]:
locations = best_weather_conditions_cities[["lat", "lon"]].astype(float)
weather_humidity = best_weather_conditions_cities["humidity"].astype(float)
maps_info_box = [
    f"<B>Hotel name</B> <BR> {city['hotel_name']} <BR> \
    <B>City</B> <BR> {city['city_name']} <BR> \
    <B>Country</B> <BR> {city['country']} <BR> \
    <B>Weather conditions</B> <BR> {city['weather_condition_desc']}" \
    for index, city in best_weather_conditions_cities.iterrows()
]

hotels_fig = gmaps.figure()

hotels_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

hotels_markers = gmaps.marker_layer(
        locations,
        info_box_content = maps_info_box
)

hotels_fig.add_layer(hotels_heat_layer)
hotels_fig.add_layer(hotels_markers)

hotels_fig

Figure(layout=FigureLayout(height='420px'))